In [149]:
from transformers import BertJapaneseTokenizer
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
max_seq_length = 1024
import re
import urllib
from urllib.request import urlopen
# 모듈 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Embedding, SimpleRNN, RNN, LSTM, GRU, Input, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import string
import tensorflow as tf

In [150]:

def list_replace(my_list):  
    for i in range(len(my_list)):
        my_list[i] = my_list[i][0]
    for i in range(len(my_list)):
        my_list[i] = my_list[i].replace("['", "")
    for i in range(len(my_list)):
        my_list[i] = my_list[i].replace("']", "")
    for i in range(len(my_list)):
        my_list[i] = my_list[i].replace('["', "")
    for i in range(len(my_list)):
        my_list[i] = my_list[i].replace('"]', "")
    for i in range(len(my_list)):
        my_list[i] = my_list[i].replace('\\n', ' ')
    return my_list

    

In [152]:
import csv


with open('1990.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    lyrics_1990 = list(csv_reader)
with open('2000.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    lyrics_2000 = list(csv_reader)
with open('2010.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    lyrics_2010 = list(csv_reader)


In [154]:


japanese_pattern = re.compile('[^\u3040-\u309F\u30A0-\u30FF\uFF00-\uFFEF\u4E00-\u9FAF]+')
lyrics_1990s=list_replace(lyrics_1990)
for i in range(len(lyrics_1990s)):
    lyrics_1990s[i] = japanese_pattern.sub('', lyrics_1990s[i])

lyrics_2000s=list_replace(lyrics_2000)
for i in range(len(lyrics_2000s)):
    lyrics_1990s[i] = japanese_pattern.sub('', lyrics_2000s[i])
    
lyrics_2010s=list_replace(lyrics_2010)
for i in range(len(lyrics_2010s)):
    lyrics_1990s[i] = japanese_pattern.sub('', lyrics_2010s[i])
    

In [155]:
tokenized_lyrics_1990s= [tokenizer.encode(text, add_special_tokens=True) for text in lyrics_1990s]
tokenized_lyrics_1990s=pad_sequences(tokenized_lyrics_1990s,maxlen=1192, padding='post', truncating='post')

tokenized_lyrics_2000s= [tokenizer.encode(text, add_special_tokens=True) for text in lyrics_2000s]
tokenized_lyrics_2000s=pad_sequences(tokenized_lyrics_2000s,maxlen=1192, padding='post', truncating='post')

tokenized_lyrics_2010s= [tokenizer.encode(text, add_special_tokens=True) for text in lyrics_2010s]
tokenized_lyrics_2010s=pad_sequences(tokenized_lyrics_2010s,maxlen=1192, padding='post', truncating='post')

Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors


In [180]:
tokenized_lyrics_2000s


array([[    2,   995,     7, ...,     0,     0,     0],
       [    2,  3328,  2606, ...,     0,     0,     0],
       [    2, 23603,    36, ...,     0,     0,     0],
       ...,
       [    2,  3654,    29, ...,     0,     0,     0],
       [    2, 22620, 23133, ...,     0,     0,     0],
       [    2, 14675,  9586, ...,     0,     0,     0]])

In [157]:

labels_1990s = [0] * len(lyrics_1990s)
labels_2000s = [1] * len(lyrics_2000s)
labels_2010s = [2] * len(lyrics_2010s)


In [158]:
import random
random.seed(42)

dataset = list(zip(tokenized_lyrics, labels))
random.shuffle(dataset)

train_size = int(0.8 * len(dataset))
train_set, test_set = dataset[:train_size], dataset[train_size:]

2775

In [178]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size ,6),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(6)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
    ])

adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


In [169]:
train_data = tf.data.Dataset.from_tensor_slices((train_set, labels))
train_data = train_data.shuffle(buffer_size=100).batch(32)

In [168]:
train_set = pad_sequences(tokenized_lyrics, maxlen=1192, dtype='int32', padding='post', truncating='post', value=0)
labels = np.asarray(labels)

assert train_set.shape[0] == labels.shape[0]

train_data = tf.data.Dataset.from_tensor_slices((train_set, labels))
train_data = train_data.shuffle(buffer_size=100).batch(32)

In [175]:
train_set.shape, labels.shape
label_oh=to_categorical(labels, num_classes=1193)

In [179]:
history = model.fit(train_set,label_oh, epochs=50)

Epoch 1/50


ValueError: in user code:

    File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Users\User\AppDat